# 使用 Ragas 评估 RAG 的检索查询效果

## 准备

In [1]:
%%time
%%capture

!pip install ragas

CPU times: user 17.5 ms, sys: 3.64 ms, total: 21.2 ms
Wall time: 3.82 s


In [1]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "天坛公园",
]

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 2.99 s, sys: 347 ms, total: 3.34 s
Wall time: 3.08 s


1

In [2]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings

Settings.llm =OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size=128
Settings.chunk_overlap=10

evaluating_llm=OpenAILike(
    model='gpt-4-turbo', 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.0,
    request_timeout=60.0
)

CPU times: user 905 ms, sys: 11.9 ms, total: 917 ms
Wall time: 916 ms


In [3]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 1.43 ms, sys: 0 ns, total: 1.43 ms
Wall time: 2.48 ms


In [4]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=documents)

CPU times: user 579 ms, sys: 44.5 ms, total: 624 ms
Wall time: 13.9 s


## 评估响应的忠实度

评估是否符合检索到的上下文，即是否不存在幻觉

### 未检索到正确结果时

In [9]:
%%time

from llama_index.core.evaluation import FaithfulnessEvaluator

query_engine = index.as_query_engine()

response = query_engine.query(
    "圆明园在哪里?"
)

evaluator = FaithfulnessEvaluator(llm=evaluating_llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result

CPU times: user 25.8 ms, sys: 138 µs, total: 25.9 ms
Wall time: 2.58 s


EvaluationResult(query=None, contexts=['收藏\n查看我的收藏\n0有用+1\n天坛公园（Temple of', '天坛公园主要建筑圜丘坛、祈年殿、皇穹宇都采用圆形平面，而祈年殿、圜丘坛的砖砌外墙则为方形，同样象征“天圆地方”。'], response='查询信息中并未提到圆明园的位置。根据提供的信息，我们只能回答与天坛公园相关的问题。因此，无法直接提供圆明园位置的答案。通常情况下，圆明园位于中国北京市海淀区，距离市中心不远。但基于当前的查询信息，我们无法给出具体的地理位置描述。', passing=False, feedback='NO', score=0.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [10]:
response.response

'查询信息中并未提到圆明园的位置。根据提供的信息，我们只能回答与天坛公园相关的问题。因此，无法直接提供圆明园位置的答案。通常情况下，圆明园位于中国北京市海淀区，距离市中心不远。但基于当前的查询信息，我们无法给出具体的地理位置描述。'

In [11]:
eval_result.passing

False

### 检索到正确结果时

In [12]:
%%time

from llama_index.core.evaluation import FaithfulnessEvaluator

query_engine = index.as_query_engine(
)

response = query_engine.query(
    "天坛在哪里?"
)

evaluator = FaithfulnessEvaluator(llm=evaluating_llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result

CPU times: user 26.1 ms, sys: 0 ns, total: 26.1 ms
Wall time: 1.66 s


EvaluationResult(query=None, contexts=['收藏\n查看我的收藏\n0有用+1\n天坛公园（Temple of', 'of Heaven），原名“天地坛”，位于北京市东城区天坛内东里7号，始建于明永乐十八年（1420年），明嘉靖九年（1530年）改名为“天坛”，是明清两代皇帝“祭天”“祈谷”的场所，总面积273公顷，是中国现存最大的古代祭祀性建筑群。'], response='天坛位于北京市东城区天坛内东里7号。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [13]:
response.response

'天坛位于北京市东城区天坛内东里7号。'

In [14]:
eval_result.passing

True

## 评估查询和响应的相关度

是否确实回答了问题。

### 未检索到正确结果时

In [15]:
%%time

from llama_index.core.evaluation import RelevancyEvaluator

evaluator = RelevancyEvaluator(llm=evaluating_llm)

query_engine = index.as_query_engine()

query = "圆明园在哪里？"
response = query_engine.query(query)

eval_result = evaluator.evaluate_response(query=query, response=response)
eval_result

CPU times: user 25.7 ms, sys: 512 µs, total: 26.2 ms
Wall time: 2.1 s


EvaluationResult(query='圆明园在哪里？', contexts=['收藏\n查看我的收藏\n0有用+1\n天坛公园（Temple of', '天坛公园主要古建筑集中于内坛，内坛中间有东西向隔墙将内坛分隔为南、北两部分，隔墙中有门相通。内坛由圜丘、祈谷坛、斋宫三组古建筑群组成。', '天坛公园主要建筑圜丘坛、祈年殿、皇穹宇都采用圆形平面，而祈年殿、圜丘坛的砖砌外墙则为方形，同样象征“天圆地方”。'], response='查询信息中并未提及圆明园的位置。根据提供的内容，我们只能回答与天坛公园相关的问题。因此，无法直接提供关于圆明园位置的答案。', passing=False, feedback='NO', score=0.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [16]:
evaluator = FaithfulnessEvaluator(llm=evaluating_llm)
eval_result = evaluator.evaluate_response(response=response)
eval_result

EvaluationResult(query=None, contexts=['收藏\n查看我的收藏\n0有用+1\n天坛公园（Temple of', '天坛公园主要古建筑集中于内坛，内坛中间有东西向隔墙将内坛分隔为南、北两部分，隔墙中有门相通。内坛由圜丘、祈谷坛、斋宫三组古建筑群组成。', '天坛公园主要建筑圜丘坛、祈年殿、皇穹宇都采用圆形平面，而祈年殿、圜丘坛的砖砌外墙则为方形，同样象征“天圆地方”。'], response='查询信息中并未提及圆明园的位置。根据提供的内容，我们只能回答与天坛公园相关的问题。因此，无法直接提供关于圆明园位置的答案。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

### 检索到正确结果时

In [17]:
%%time

from llama_index.core.evaluation import RelevancyEvaluator

evaluator = RelevancyEvaluator(llm=evaluating_llm)

query_engine = index.as_query_engine()

query = "天坛在哪里？"
response = query_engine.query(query)

eval_result = evaluator.evaluate_response(query=query, response=response)
eval_result

CPU times: user 20.9 ms, sys: 4.11 ms, total: 25 ms
Wall time: 1.65 s


EvaluationResult(query='天坛在哪里？', contexts=['收藏\n查看我的收藏\n0有用+1\n天坛公园（Temple of', 'of Heaven），原名“天地坛”，位于北京市东城区天坛内东里7号，始建于明永乐十八年（1420年），明嘉靖九年（1530年）改名为“天坛”，是明清两代皇帝“祭天”“祈谷”的场所，总面积273公顷，是中国现存最大的古代祭祀性建筑群。'], response='天坛位于北京市东城区天坛内东里7号。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

## 评估回答和参考答案的的相关性

### 准确的情况

In [19]:
%%time

from llama_index.core.evaluation import CorrectnessEvaluator

evaluator = CorrectnessEvaluator(llm=evaluating_llm)

query = "天坛在哪里？"
reference="天坛公园位于北京市东城区天坛内东里7号。"
response = query_engine.query(query)


result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=reference,
)

result

CPU times: user 24.3 ms, sys: 0 ns, total: 24.3 ms
Wall time: 5.97 s


EvaluationResult(query='天坛在哪里？', contexts=None, response='天坛位于北京市东城区天坛内东里7号。', passing=True, feedback='The generated answer correctly addresses the user query by providing the precise location of the Temple of Heaven, which matches the reference answer. It is relevant and accurate.', score=5.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

In [20]:
response.response

'天坛位于北京市东城区天坛内东里7号。'

In [21]:
result.score

5.0

### 不准确的情况

In [22]:
%%time

from llama_index.core.evaluation import CorrectnessEvaluator

evaluator = CorrectnessEvaluator(llm=evaluating_llm)

query = "颐和园在哪里？"
reference="颐和园坐落在北京西郊。"
response = query_engine.query(query)


result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=reference,
)

result

CPU times: user 21.1 ms, sys: 2.59 ms, total: 23.7 ms
Wall time: 8.01 s


EvaluationResult(query='颐和园在哪里？', contexts=None, response='查询信息中没有提到颐和园的位置，因此无法提供关于颐和园位置的答案。', passing=False, feedback='The generated answer does not provide the location of the Summer Palace, which was the exact information requested in the user query. The answer is completely irrelevant as it claims that there is no information on the location, which is incorrect. The correct location should be that the Summer Palace (颐和园) is located in the western suburbs of Beijing, as provided in the reference answer.', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)